#### Simple notebook to run model tests only

In [30]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor

In [10]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

## Catboost

In [7]:
#Create a pool of data
train_pool = Pool(X_train, y_train, cat_features=["location"])
test_pool = Pool(X_test, cat_features=["location"]) 

cat_feature = ["location"]

#init model and fit it
catboost_model = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", cat_features=cat_feature, verbose=100)
catboost_model.fit(train_pool)
print("")

0:	learn: 285.2960531	total: 306ms	remaining: 5m 5s
100:	learn: 94.0535113	total: 30.5s	remaining: 4m 31s
200:	learn: 85.4838123	total: 1m 2s	remaining: 4m 8s
300:	learn: 82.9398598	total: 1m 33s	remaining: 3m 36s
400:	learn: 80.8394103	total: 2m 6s	remaining: 3m 9s
500:	learn: 78.9044892	total: 2m 40s	remaining: 2m 40s
600:	learn: 76.9682906	total: 3m 14s	remaining: 2m 9s
700:	learn: 75.3143619	total: 3m 47s	remaining: 1m 37s
800:	learn: 73.8116012	total: 4m 22s	remaining: 1m 5s
900:	learn: 72.0202153	total: 4m 58s	remaining: 32.8s
999:	learn: 70.6792914	total: 5m 33s	remaining: 0us



In [26]:
catboost_model.get_feature_importance()

array([5.84361316e-01, 4.17091513e-01, 2.96880145e-01, 2.43437044e+00,
       4.32688271e+00, 5.62287811e-01, 1.06781780e-04, 6.55233426e-01,
       1.77960798e+00, 5.68483469e-01, 1.25757987e+01, 3.35238284e+00,
       4.24354725e+00, 2.70343617e+01, 5.76731375e-03, 1.55130847e-05,
       2.43668225e-01, 2.87369588e-04, 9.47068259e-03, 1.70520459e-02,
       1.12954553e-01, 4.31647511e-01, 2.51294488e-01, 4.38010946e-01,
       2.62406292e-01, 2.90149046e-01, 3.17302370e-04, 2.71996969e-02,
       1.73608779e+00, 2.56813887e-01, 2.91341776e-02, 0.00000000e+00,
       3.76342692e-03, 2.19281392e-01, 1.29732337e+00, 6.03619154e+00,
       2.32270574e-01, 9.61655907e-01, 9.59912491e-01, 6.93837093e-01,
       5.34365778e-01, 7.48165307e-01, 5.07625357e-01, 6.63314403e-04,
       1.24189301e-01, 2.03899539e+01, 1.55471300e+00, 6.08523329e-01,
       1.06854582e+00, 1.11534606e+00])

In [27]:
catboost_model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'LogCosh',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Exact',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderT

#### Catboost for A and (B and C)

#### Local Test for all data in 1 dataset

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
ts = TimeSeriesSplit(n_splits = 10)
sklearn_cv_result = cross_val_score(catboost_model, X_train, y_train, cv=ts, scoring='neg_mean_absolute_error')

NameError: name 'catboost_model' is not defined

In [15]:
np.mean(sklearn_cv_result)

-153.76774856671472

In [8]:
#make predictions
def replace_under_0_2(x):
    if x < 0.2:
        return 0
    else:
        return x

predictions = pd.DataFrame(catboost_model.predict(test_pool))
predictions = predictions.clip(lower=0)
predictions = predictions.applymap(replace_under_0_2)
print(predictions)

             0
0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
...        ...
8635  0.269505
8636  0.274530
8637  0.310932
8638  0.000000
8639  0.000000

[8640 rows x 1 columns]


In [36]:
1/cv_results['test-RMSE-mean'].min()

0.0029318377232400933

## Setting up local validation set

In [99]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2020-05-01') & (df.index <= '2020-07-25')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_trainnew_a, X_test_new_a, y_train_new_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

X_train_loc_a, X_test_loc_a, y_train_loc_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"], y_train_a], axis=1))
X_train_loc_b, X_test_loc_b, y_train_loc_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"], y_train_b], axis=1))
X_train_loc_c, X_test_loc_c, y_train_loc_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"], y_train_c], axis=1))


X_train_new = pd.concat([X_train_loc_a, X_train_loc_b, X_train_loc_c])
X_test_new = pd.concat([X_test_loc_a, X_test_loc_b, X_test_loc_c])
y_train_new = pd.concat([y_train_loc_a, y_train_loc_b, y_train_loc_c])
y_test = pd.concat([y_test_a, y_test_b, y_test_c])


In [64]:
X_train_new.to_csv("X_train_new.csv")
X_test_new.to_csv("X_test_new.csv")

In [33]:
X_test.index

DatetimeIndex(['2023-05-01 00:00:00', '2023-05-01 00:15:00',
               '2023-05-01 00:30:00', '2023-05-01 00:45:00',
               '2023-05-01 01:00:00', '2023-05-01 01:15:00',
               '2023-05-01 01:30:00', '2023-05-01 01:45:00',
               '2023-05-01 02:00:00', '2023-05-01 02:15:00',
               ...
               '2023-07-03 21:30:00', '2023-07-03 21:45:00',
               '2023-07-03 22:00:00', '2023-07-03 22:15:00',
               '2023-07-03 22:30:00', '2023-07-03 22:45:00',
               '2023-07-03 23:00:00', '2023-07-03 23:15:00',
               '2023-07-03 23:30:00', '2023-07-03 23:45:00'],
              dtype='datetime64[us]', name='date_forecast', length=8640, freq=None)

In [59]:
X_train[X_train["estimated"] == 1].index

DatetimeIndex(['2022-10-28 22:00:00', '2022-10-28 23:00:00',
               '2022-10-29 00:00:00', '2022-10-29 01:00:00',
               '2022-10-29 02:00:00', '2022-10-29 03:00:00',
               '2022-10-29 04:00:00', '2022-10-29 05:00:00',
               '2022-10-29 06:00:00', '2022-10-29 07:00:00',
               ...
               '2023-04-30 14:00:00', '2023-04-30 15:00:00',
               '2023-04-30 16:00:00', '2023-04-30 17:00:00',
               '2023-04-30 18:00:00', '2023-04-30 19:00:00',
               '2023-04-30 20:00:00', '2023-04-30 21:00:00',
               '2023-04-30 22:00:00', '2023-04-30 23:00:00'],
              dtype='datetime64[us]', length=10925, freq=None)

In [15]:
#Create a pool of data
params_a = {'iterations': 2815, 'learning_rate': 0.007099215379206612, 'depth': 13, 'min_data_in_leaf': 84, 'l2_leaf_reg': 2, 'bagging_temperature': 0.5343409157539628, 'random_strength': 0.7466589469065493, 'border_count': 785, 'rsm': 0.3011005019052161}
params_b = {'iterations': 2569, 'learning_rate': 0.004964798577466536, 'depth': 3, 'min_data_in_leaf': 34, 'l2_leaf_reg': 8, 'bagging_temperature': 0.7465339246522942, 'random_strength': 0.6335246931927535, 'border_count': 80, 'rsm': 0.43150327625887724}
params_c = {'iterations': 2150, 'learning_rate': 0.024455378511188973, 'depth': 7, 'min_data_in_leaf': 10, 'l2_leaf_reg': 9, 'bagging_temperature': 0.6897741884012124, 'random_strength': 0.579016268195538, 'border_count': 539, 'rsm': 0.5554441757810058}

train_pool_a = Pool(X_train_new_a, y_train_new_a)
train_pool_b = Pool(X_train_new_b, y_train_new_b)
train_pool_c = Pool(X_train_new_c, y_train_new_c)


test_pool_a = Pool(X_test_new_a) 
test_pool_b = Pool(X_test_new_b) 
test_pool_c = Pool(X_test_new_c) 



#init model and fit it
catboost_model_a = CatBoostRegressor(**params_a, verbose=100)
catboost_model_b = CatBoostRegressor(**params_b, verbose=100)
catboost_model_c = CatBoostRegressor(**params_c, verbose=100)

catboost_model_a.fit(train_pool_a)
catboost_model_b.fit(train_pool_b)
catboost_model_c.fit(train_pool_c)

0:	learn: 1133.1333463	total: 833ms	remaining: 39m 3s
100:	learn: 696.7460829	total: 49.3s	remaining: 22m 5s
200:	learn: 511.2636395	total: 1m 38s	remaining: 21m 27s
300:	learn: 431.3117068	total: 2m 29s	remaining: 20m 44s
400:	learn: 393.1878447	total: 3m 18s	remaining: 19m 56s
500:	learn: 370.4534957	total: 4m 8s	remaining: 19m 6s
600:	learn: 353.8886965	total: 4m 59s	remaining: 18m 23s
700:	learn: 338.8204651	total: 5m 50s	remaining: 17m 37s
800:	learn: 326.1606564	total: 6m 40s	remaining: 16m 47s
900:	learn: 314.3331653	total: 7m 30s	remaining: 15m 56s
1000:	learn: 302.6318073	total: 8m 22s	remaining: 15m 11s
1100:	learn: 292.5123214	total: 9m 12s	remaining: 14m 20s
1200:	learn: 282.9863647	total: 10m 2s	remaining: 13m 29s
1300:	learn: 273.2301665	total: 10m 52s	remaining: 12m 39s
1400:	learn: 264.7931475	total: 11m 43s	remaining: 11m 49s
1500:	learn: 256.5474600	total: 12m 33s	remaining: 10m 59s
1600:	learn: 248.7696180	total: 13m 23s	remaining: 10m 9s
1700:	learn: 241.1792551	tot

In [8]:
catboost_model_b.get_all_params()

CatBoostError: There is no trained model to use get_all_params(). Use fit() to train model. Then use this method.

In [18]:
pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))

In [19]:
from sklearn.metrics import mean_absolute_error
MAE_a = mean_absolute_error(y_test_a, pred_a)
MAE_b = mean_absolute_error(y_test_b, pred_b)
MAE_c = mean_absolute_error(y_test_c, pred_c)

print(np.mean([MAE_a, MAE_b, MAE_c]))

152.58244977271463


In [13]:
#final training and predictions
params_a = {'iterations': 2511, 'learning_rate': 0.013387708538234228, 'depth': 9, 'min_data_in_leaf': 37, 'l2_leaf_reg': 9, 'bagging_temperature': 0.6037942407543951, 'random_strength': 0.7203046909719719, 'border_count': 792, 'rsm': 0.5142593972884377, "loss_function": "LogCosh"}
params_b = {'iterations': 1647, 'learning_rate': 0.008285667949530987, 'depth': 2, 'min_data_in_leaf': 7, 'l2_leaf_reg': 6, 'bagging_temperature': 0.638344903820208, 'random_strength': 0.9388444992830671, 'border_count': 434, 'rsm': 0.8347735297026142, "loss_function": "LogCosh"}
params_c = {'iterations': 2216, 'learning_rate': 0.02932152249119453, 'depth': 10, 'min_data_in_leaf': 21, 'l2_leaf_reg': 2, 'bagging_temperature': 0.8784102456931138, 'random_strength': 0.6901129919784297, 'border_count': 119, 'rsm': 0.8571604856673344, "loss_function": "LogCosh"}

X_train_a = X_train[X_train["location"] == "A"].drop("location", axis=1)
X_train_b = X_train[X_train["location"] == "B"].drop("location", axis=1)
X_train_c = X_train[X_train["location"] == "C"].drop("location", axis=1)

X_test_a = X_test[X_test["location"] == "A"].drop("location", axis=1)
X_test_b = X_test[X_test["location"] == "B"].drop("location", axis=1)
X_test_c = X_test[X_test["location"] == "C"].drop("location", axis=1)
          
train_pool_a = Pool(X_train_a, y_train_a)
train_pool_b = Pool(X_train_b, y_train_b)
train_pool_c = Pool(X_train_c, y_train_c)


test_pool_a = Pool(X_test_a) 
test_pool_b = Pool(X_test_b) 
test_pool_c = Pool(X_test_c) 

catboost_model_a = CatBoostRegressor(**params_a)
catboost_model_b = CatBoostRegressor(**params_b)
catboost_model_c = CatBoostRegressor(**params_c)

catboost_model_a.fit(train_pool_a)
catboost_model_b.fit(train_pool_b)
catboost_model_c.fit(train_pool_c)


pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))

0:	learn: 624.9320516	total: 81.5ms	remaining: 3m 24s
1:	learn: 617.8554232	total: 189ms	remaining: 3m 57s
2:	learn: 612.3991148	total: 294ms	remaining: 4m 5s
3:	learn: 605.8252095	total: 398ms	remaining: 4m 9s
4:	learn: 599.7155338	total: 516ms	remaining: 4m 18s
5:	learn: 593.2841907	total: 629ms	remaining: 4m 22s
6:	learn: 586.9708285	total: 732ms	remaining: 4m 21s
7:	learn: 581.1871917	total: 839ms	remaining: 4m 22s
8:	learn: 574.6402089	total: 934ms	remaining: 4m 19s
9:	learn: 568.2828759	total: 1.04s	remaining: 4m 20s
10:	learn: 562.9091955	total: 1.17s	remaining: 4m 25s
11:	learn: 556.2755672	total: 1.25s	remaining: 4m 20s
12:	learn: 550.5359757	total: 1.35s	remaining: 4m 18s
13:	learn: 545.2325326	total: 1.41s	remaining: 4m 11s
14:	learn: 539.2937632	total: 1.49s	remaining: 4m 8s
15:	learn: 533.0986570	total: 1.59s	remaining: 4m 8s
16:	learn: 527.5538033	total: 1.69s	remaining: 4m 7s
17:	learn: 521.5345024	total: 1.77s	remaining: 4m 5s
18:	learn: 515.5937522	total: 1.85s	remaini

In [14]:
def replace_under_0_2(x):
    if x < 0.2:
        return 0
    else:
        return x

predictions = pd.DataFrame(pd.concat([pred_a, pred_b, pred_c]))
predictions = predictions.clip(lower=0)
predictions = predictions.applymap(replace_under_0_2)

#### validation on single dataset

In [100]:
train_pool = Pool(X_train_new, y_train_new, cat_features=["location"])
test_pool = Pool(X_test_new, cat_features=["location"]) 


#init model and fit it
catboost_model_val = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100)
catboost_model_val.fit(train_pool)

0:	learn: 256.8930350	total: 313ms	remaining: 5m 12s
100:	learn: 90.3088700	total: 30.1s	remaining: 4m 28s
200:	learn: 81.9604132	total: 51s	remaining: 3m 22s
300:	learn: 79.2658502	total: 1m 6s	remaining: 2m 33s
400:	learn: 77.2022654	total: 1m 21s	remaining: 2m 1s
500:	learn: 75.0828857	total: 1m 36s	remaining: 1m 36s
600:	learn: 73.3366229	total: 1m 52s	remaining: 1m 14s
700:	learn: 71.9246621	total: 2m 7s	remaining: 54.6s
800:	learn: 70.5370289	total: 2m 24s	remaining: 35.8s
900:	learn: 69.3552203	total: 2m 40s	remaining: 17.6s
999:	learn: 68.4802220	total: 2m 56s	remaining: 0us


In [101]:
from sklearn.metrics import mean_absolute_error
pred = pd.DataFrame(catboost_model_val.predict(test_pool))
MAE = mean_absolute_error(y_test, pred)
print(MAE)

148.76581941974032


In [25]:
#final prediction
cat_feature = ["location"]
train_pool = Pool(X_train, y_train, cat_features=cat_feature)
test_pool  = Pool(X_test, cat_features=cat_feature) 

CatBoostRegressor(iterations=1000, depth=9, loss_function="MAE")
catboost_model.fit(train_pool)

0:	learn: 281.8289325	total: 98.7ms	remaining: 1m 38s
1:	learn: 275.2872644	total: 191ms	remaining: 1m 35s
2:	learn: 269.4017188	total: 288ms	remaining: 1m 35s
3:	learn: 263.0293105	total: 392ms	remaining: 1m 37s
4:	learn: 257.7050440	total: 490ms	remaining: 1m 37s
5:	learn: 253.0703718	total: 585ms	remaining: 1m 36s
6:	learn: 247.3628511	total: 679ms	remaining: 1m 36s
7:	learn: 242.0504438	total: 778ms	remaining: 1m 36s
8:	learn: 236.8711053	total: 882ms	remaining: 1m 37s
9:	learn: 234.0909253	total: 983ms	remaining: 1m 37s
10:	learn: 228.4708410	total: 1.08s	remaining: 1m 37s
11:	learn: 223.0565392	total: 1.19s	remaining: 1m 37s
12:	learn: 220.1855277	total: 1.28s	remaining: 1m 37s
13:	learn: 215.1868880	total: 1.38s	remaining: 1m 37s
14:	learn: 210.4037898	total: 1.48s	remaining: 1m 37s
15:	learn: 205.6795086	total: 1.58s	remaining: 1m 37s
16:	learn: 201.3969569	total: 1.68s	remaining: 1m 37s
17:	learn: 197.0129786	total: 1.78s	remaining: 1m 37s
18:	learn: 192.8338636	total: 1.89s	r

In [122]:
predictions = pd.DataFrame(catboost_model.predict(test_pool))

## Setting up Catboost for stacking

In [31]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train_avg = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train_avg = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a_avg = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b_avg = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c_avg = pd.read_parquet('data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [32]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train_non_avg = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train_non_avg = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a_non_avg = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b_non_avg = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c_non_avg = pd.read_parquet('data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [33]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/avg/summer/X_test.parquet')
X_train_avg_summer = pd.read_parquet('data/prepared_datasets/avg/summer/X_train.parquet')
y_train_avg_summer = pd.read_parquet('data/prepared_datasets/avg/summer/Y_train.parquet')
y_train_a_avg_summer = pd.read_parquet('data/prepared_datasets/avg/summer/Y_train_a.parquet')
y_train_b_avg_summer = pd.read_parquet('data/prepared_datasets/avg/summer/Y_train_b.parquet')
y_train_c_avg_summer = pd.read_parquet('data/prepared_datasets/avg/summer/Y_train_c.parquet')

In [34]:
#Load inn datasets
X_test  = pd.read_parquet('data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train_non_avg_summer = pd.read_parquet('data/prepared_datasets/summer/X_train.parquet')
y_train_non_avg_summer = pd.read_parquet('data/prepared_datasets/summer/Y_train.parquet')
y_train_a_non_avg_summer = pd.read_parquet('data/prepared_datasets/summer/Y_train_a.parquet')
y_train_b_non_avg_summer = pd.read_parquet('data/prepared_datasets/summer/Y_train_b.parquet')
y_train_c_non_avg_summer = pd.read_parquet('data/prepared_datasets/summer/Y_train_c.parquet')

In [35]:
#Create a pool of data
train_pool_avg = Pool(X_train_avg, y_train_avg, cat_features=["location"])
test_pool_avg = Pool(X_test, cat_features=["location"]) 

train_pool_non_avg = Pool(X_train_non_avg, y_train_non_avg, cat_features=["location"])
test_pool_non_avg = Pool(X_test, cat_features=["location"]) 

#init models and fit them
catboost_model_avg = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100)
catboost_model_avg.fit(train_pool_avg)

catboost_model_non_avg = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100)
catboost_model_non_avg.fit(train_pool_non_avg)

0:	learn: 288.4237594	total: 175ms	remaining: 2m 55s
100:	learn: 95.0993860	total: 18.7s	remaining: 2m 46s
200:	learn: 85.7607295	total: 37.6s	remaining: 2m 29s
300:	learn: 83.1566183	total: 57.3s	remaining: 2m 13s
400:	learn: 81.0675620	total: 1m 17s	remaining: 1m 55s
500:	learn: 79.1934025	total: 1m 36s	remaining: 1m 36s
600:	learn: 77.3369244	total: 1m 56s	remaining: 1m 17s
700:	learn: 75.6892747	total: 2m 16s	remaining: 58.2s
800:	learn: 73.9179017	total: 2m 36s	remaining: 38.9s
900:	learn: 72.2539595	total: 2m 56s	remaining: 19.4s
999:	learn: 70.8529308	total: 3m 16s	remaining: 0us
0:	learn: 280.0059853	total: 205ms	remaining: 3m 25s
100:	learn: 99.0865828	total: 19.2s	remaining: 2m 50s
200:	learn: 90.2153171	total: 38.7s	remaining: 2m 33s
300:	learn: 87.5726426	total: 58.5s	remaining: 2m 15s
500:	learn: 83.7731682	total: 1m 38s	remaining: 1m 38s
600:	learn: 82.4051164	total: 1m 59s	remaining: 1m 19s
700:	learn: 81.1314369	total: 2m 19s	remaining: 59.6s
800:	learn: 79.6476444	tota

In [36]:
#Fit catboost models with hyperparamaters tuned for different summers - avg

#Create a pool of data
train_pool_avg = Pool(X_train_avg, y_train_avg, cat_features=["location"])
test_pool_avg = Pool(X_test, cat_features=["location"]) 

params_2020 = {'iterations': 2420, 'learning_rate': 0.009233314270894907, 'depth': 10, 'min_data_in_leaf': 80, 'l2_leaf_reg': 2, 'bagging_temperature': 0.6243644480938084, 'random_strength': 0.338232262367468, 'border_count': 998, 'rsm': 0.49730615451830307, "loss_function": "LogCosh"}
params_2021 = {'iterations': 1382, 'learning_rate': 0.01728020929424134, 'depth': 8, 'min_data_in_leaf': 55, 'l2_leaf_reg': 5, 'bagging_temperature': 0.8030642689284688, 'random_strength': 0.46958583091012845, 'border_count': 253, 'rsm': 0.8339839237916834, "loss_function": "LogCosh"}

#init models and fit them
catboost_model_avg_2020 = CatBoostRegressor(**params_2020, verbose=100)
catboost_model_avg_2020.fit(train_pool_avg)

catboost_model_avg_2021 = CatBoostRegressor(**params_2021, verbose=100)
catboost_model_avg_2021.fit(train_pool_avg)

0:	learn: 290.3963505	total: 193ms	remaining: 7m 45s
100:	learn: 153.1334326	total: 20.9s	remaining: 7m 59s
200:	learn: 109.0990472	total: 42.2s	remaining: 7m 46s
300:	learn: 94.0873700	total: 1m 3s	remaining: 7m 25s
400:	learn: 87.1559915	total: 1m 25s	remaining: 7m 9s
500:	learn: 84.1906891	total: 1m 46s	remaining: 6m 49s
600:	learn: 82.6315426	total: 2m 9s	remaining: 6m 30s
700:	learn: 81.3515862	total: 2m 31s	remaining: 6m 10s
800:	learn: 80.3052599	total: 2m 52s	remaining: 5m 49s
900:	learn: 79.4737748	total: 3m 14s	remaining: 5m 28s
1000:	learn: 78.7413061	total: 3m 36s	remaining: 5m 7s
1100:	learn: 78.1355180	total: 3m 58s	remaining: 4m 45s
1200:	learn: 77.5834435	total: 4m 21s	remaining: 4m 25s
1300:	learn: 76.8468662	total: 4m 43s	remaining: 4m 3s
1400:	learn: 76.0774350	total: 5m 6s	remaining: 3m 43s
1500:	learn: 75.3482665	total: 5m 29s	remaining: 3m 21s
1600:	learn: 74.6663614	total: 5m 51s	remaining: 2m 59s
1700:	learn: 73.9263509	total: 6m 13s	remaining: 2m 38s
1800:	lear

In [72]:
params_non_avg_2021 = {'iterations': 1635, 'learning_rate': 0.019802005526336503, 'depth': 9, 'min_data_in_leaf': 73, 'l2_leaf_reg': 5, 'bagging_temperature': 0.8076037123638955, 'random_strength': 0.9831381174553756, 'border_count': 352, 'rsm': 0.627289715050461, "loss_function" : "LogCosh"}
params_non_avg_2020 = {'iterations': 2714, 'learning_rate': 0.0034852141379466154, 'depth': 10, 'min_data_in_leaf': 59, 'l2_leaf_reg': 8, 'bagging_temperature': 0.38395681407227494, 'random_strength': 0.4673864208314273, 'border_count': 864, 'rsm': 0.5408044477854796, "loss_function" : "LogCosh"}

#init models and fit them
catboost_model_non_avg_2020 = CatBoostRegressor(**params_non_avg_2020, verbose=100)
catboost_model_non_avg_2020.fit(train_pool_avg)

catboost_model_non_avg_2021 = CatBoostRegressor(**params_non_avg_2021, verbose=100)
catboost_model_non_avg_2021.fit(train_pool_avg)

0:	learn: 291.4630175	total: 128ms	remaining: 5m 46s
100:	learn: 222.7919817	total: 13s	remaining: 5m 37s
200:	learn: 174.0984143	total: 26.2s	remaining: 5m 27s
300:	learn: 145.0706611	total: 39.6s	remaining: 5m 17s
400:	learn: 126.0451337	total: 53.3s	remaining: 5m 7s
500:	learn: 112.9986758	total: 1m 6s	remaining: 4m 55s
600:	learn: 104.3835733	total: 1m 20s	remaining: 4m 44s
700:	learn: 98.7831989	total: 1m 34s	remaining: 4m 31s
800:	learn: 94.2483517	total: 1m 48s	remaining: 4m 19s
900:	learn: 90.9186761	total: 2m 2s	remaining: 4m 6s
1000:	learn: 88.6108971	total: 2m 16s	remaining: 3m 53s
1100:	learn: 86.9770857	total: 2m 30s	remaining: 3m 40s
1200:	learn: 85.7550591	total: 2m 44s	remaining: 3m 27s
1300:	learn: 84.8367867	total: 2m 59s	remaining: 3m 14s
1400:	learn: 84.0517911	total: 3m 13s	remaining: 3m 1s
1500:	learn: 83.4976867	total: 3m 27s	remaining: 2m 47s
1600:	learn: 82.9912106	total: 3m 42s	remaining: 2m 34s
1700:	learn: 82.4592590	total: 3m 56s	remaining: 2m 20s
1800:	lea

In [37]:
#Create a pool of data
train_pool_avg_summer = Pool(X_train_avg_summer, y_train_avg_summer, cat_features=["location"])
test_pool_avg_summer = Pool(X_test, cat_features=["location"]) 

train_pool_non_avg_summer = Pool(X_train_non_avg_summer, y_train_non_avg_summer, cat_features=["location"])
test_pool_non_avg_summer = Pool(X_test, cat_features=["location"]) 

#init models and fit them
catboost_model_avg_summer = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100)
catboost_model_avg_summer.fit(train_pool_avg_summer)

catboost_model_non_avg_summer = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100)
catboost_model_non_avg_summer.fit(train_pool_non_avg_summer)

0:	learn: 288.4237594	total: 215ms	remaining: 3m 34s
100:	learn: 95.0993860	total: 20.7s	remaining: 3m 4s
200:	learn: 85.7607295	total: 42.2s	remaining: 2m 47s
300:	learn: 83.1566183	total: 1m 4s	remaining: 2m 28s
400:	learn: 81.0675620	total: 1m 25s	remaining: 2m 8s
500:	learn: 79.1934025	total: 1m 48s	remaining: 1m 48s
600:	learn: 77.3369244	total: 2m 11s	remaining: 1m 26s
700:	learn: 75.6892747	total: 2m 33s	remaining: 1m 5s
800:	learn: 73.9179017	total: 2m 55s	remaining: 43.7s
900:	learn: 72.2539595	total: 3m 18s	remaining: 21.8s
999:	learn: 70.8529308	total: 3m 40s	remaining: 0us
0:	learn: 639.7348129	total: 59ms	remaining: 59s
100:	learn: 188.2525414	total: 6.8s	remaining: 1m
200:	learn: 165.1857390	total: 13.4s	remaining: 53.4s
300:	learn: 157.5460833	total: 20.4s	remaining: 47.3s
400:	learn: 150.8129488	total: 27.4s	remaining: 41s
500:	learn: 145.4327217	total: 34.6s	remaining: 34.4s
600:	learn: 140.0883557	total: 41.6s	remaining: 27.6s
700:	learn: 136.0130353	total: 48.8s	rema

In [46]:
#create training location per location with prev found hyperparamaters focusing on 2020 summer
params_a = {'iterations': 2511, 'learning_rate': 0.013387708538234228, 'depth': 9, 'min_data_in_leaf': 37, 'l2_leaf_reg': 9, 'bagging_temperature': 0.6037942407543951, 'random_strength': 0.7203046909719719, 'border_count': 792, 'rsm': 0.5142593972884377, "loss_function": "LogCosh"}
params_b = {'iterations': 1647, 'learning_rate': 0.008285667949530987, 'depth': 2, 'min_data_in_leaf': 7, 'l2_leaf_reg': 6, 'bagging_temperature': 0.638344903820208, 'random_strength': 0.9388444992830671, 'border_count': 434, 'rsm': 0.8347735297026142, "loss_function": "LogCosh"}
params_c = {'iterations': 2216, 'learning_rate': 0.02932152249119453, 'depth': 10, 'min_data_in_leaf': 21, 'l2_leaf_reg': 2, 'bagging_temperature': 0.8784102456931138, 'random_strength': 0.6901129919784297, 'border_count': 119, 'rsm': 0.8571604856673344, "loss_function": "LogCosh"}

X_train_a = X_train_non_avg[X_train_non_avg["location"] == "A"].drop("location", axis=1)
X_train_b = X_train_non_avg[X_train_non_avg["location"] == "B"].drop("location", axis=1)
X_train_c = X_train_non_avg[X_train_non_avg["location"] == "C"].drop("location", axis=1)

X_test_a = X_test[X_test["location"] == "A"].drop("location", axis=1)
X_test_b = X_test[X_test["location"] == "B"].drop("location", axis=1)
X_test_c = X_test[X_test["location"] == "C"].drop("location", axis=1)
          
train_pool_a = Pool(X_train_a, y_train_a_non_avg)
train_pool_b = Pool(X_train_b, y_train_b_non_avg)
train_pool_c = Pool(X_train_c, y_train_c_non_avg)


test_pool_a = Pool(X_test_a) 
test_pool_b = Pool(X_test_b) 
test_pool_c = Pool(X_test_c) 

catboost_model_a = CatBoostRegressor(**params_a)
catboost_model_b = CatBoostRegressor(**params_b)
catboost_model_c = CatBoostRegressor(**params_c)

catboost_model_a.fit(train_pool_a)
catboost_model_b.fit(train_pool_b)
catboost_model_c.fit(train_pool_c)


pred_a = catboost_model_a.predict(test_pool_a)
pred_b = catboost_model_b.predict(test_pool_b)
pred_c = catboost_model_c.predict(test_pool_c)

0:	learn: 624.9320516	total: 53.5ms	remaining: 2m 14s
1:	learn: 617.8554232	total: 158ms	remaining: 3m 18s
2:	learn: 612.3991148	total: 205ms	remaining: 2m 51s
3:	learn: 605.8252095	total: 261ms	remaining: 2m 43s
4:	learn: 599.7155338	total: 319ms	remaining: 2m 40s
5:	learn: 593.2841907	total: 430ms	remaining: 2m 59s
6:	learn: 586.9708285	total: 476ms	remaining: 2m 50s
7:	learn: 581.1871917	total: 529ms	remaining: 2m 45s
8:	learn: 574.6402089	total: 583ms	remaining: 2m 41s
9:	learn: 568.2828759	total: 641ms	remaining: 2m 40s
10:	learn: 562.9091955	total: 716ms	remaining: 2m 42s
11:	learn: 556.2755672	total: 767ms	remaining: 2m 39s
12:	learn: 550.5359757	total: 823ms	remaining: 2m 38s
13:	learn: 545.2325326	total: 880ms	remaining: 2m 36s
14:	learn: 539.2937632	total: 956ms	remaining: 2m 39s
15:	learn: 533.0986570	total: 1.01s	remaining: 2m 37s
16:	learn: 527.5538033	total: 1.07s	remaining: 2m 37s
17:	learn: 521.5345024	total: 1.13s	remaining: 2m 36s
18:	learn: 515.5937522	total: 1.23s	r

In [65]:
pred_a = catboost_model_a.predict(test_pool_a)
pred_b = catboost_model_b.predict(test_pool_b)
pred_c = catboost_model_c.predict(test_pool_c)
pred_non_avg_per_location = np.concatenate((pred_a, pred_b, pred_c))

pred_non_avg_per_location[pred_non_avg_per_location < 0] = 0

In [73]:
#Make predictions for the different validation sets - non avg
pred_non_avg_2020 = catboost_model_non_avg_2020.predict(test_pool_avg)
pred_non_avg_2021 = catboost_model_non_avg_2021.predict(test_pool_avg)
pred_non_avg_2020[pred_non_avg_2020 < 0] = 0
pred_non_avg_2021[pred_non_avg_2021 < 0] = 0

pred_stacked_non_avg_2020_2021 = (pred_non_avg_2020 + pred_non_avg_2021) / 2

In [59]:
print(pred_non_avg_per_location)

[0.03855167 0.04342781 0.04262114 ... 0.10524848 0.         0.        ]


In [66]:
#Make predictions for the different validation sets
pred_avg_2020 = catboost_model_avg_2020.predict(test_pool_avg)
pred_avg_2021 = catboost_model_avg_2021.predict(test_pool_avg)
pred_avg_2020[pred_avg_2020 < 0] = 0
pred_avg_2021[pred_avg_2021 < 0] = 0

pred_stacked_avg_2020_2021 = (pred_avg_2020 + pred_avg_2021) / 2

In [64]:
#Make predictions for summer
pred_avg_summer = catboost_model_avg_summer.predict(test_pool_avg_summer)
pred_non_avg_summer = catboost_model_non_avg_summer.predict(test_pool_non_avg_summer)

pred_stacked_summer = (pred_avg_summer + pred_non_avg_summer) / 2
dfdfdfdf = pd.DataFrame(pred_stacked_summer).to_csv("onlysummer.csv")

In [67]:
#Make predictions for non-summer
pred_avg = catboost_model_avg.predict(test_pool_avg)
pred_non_avg = catboost_model_non_avg.predict(test_pool_non_avg)
pred_avg[pred_avg < 0] = 0
pred_non_avg[pred_non_avg < 0] = 0

pred_stacked = (pred_avg + pred_non_avg) / 2

In [74]:
pred_tot_stacked = pd.DataFrame((pred_stacked + pred_stacked_avg_2020_2021 + pred_non_avg_per_location + pred_stacked_non_avg_2020_2021)/4)

predictions = pred_tot_stacked.clip(lower=0)

def replace_under_0_2(x):
    if x < 0.2:
        return 0
    else:
        return x
    
predictions = pred_tot_stacked.applymap(replace_under_0_2)

## Preparing submission

In [75]:
def prepare_submission(predictions, X_test):
    index_df = X_test.index.to_frame()
    out_pd = pd.concat([index_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)
    out_pd = out_pd.rename(columns={0: 'prediction', 'date_forecast': 'time'})
    print(X_test.info())
    print(out_pd.info())
    out_pd['location'] = X_test['location'].reset_index(drop=True)
    out_pd.set_index('time', inplace=True)
    return out_pd

def merge_with_sample(out_pd):
    test = pd.read_csv('data/test.csv')
    test.time = pd.to_datetime(test.time)
    sample_submission = pd.read_csv('data/sample_submission.csv')
    test.set_index('time', inplace=True)
    
    merged_df = test.reset_index().merge(out_pd.reset_index(), on=['time', 'location'], how='left', suffixes=('_original', '_new'))
    merged_df['prediction_new'] = merged_df['prediction_new'].combine_first(merged_df['prediction_original'])
    merged_df.drop('prediction_original', axis=1, inplace=True)
    merged_df.rename(columns={'prediction_new': 'prediction'}, inplace=True)
    return sample_submission[['id']].merge(merged_df[['id', 'prediction']], on='id', how='left')


out_pd = prepare_submission(predictions, X_test)
sample_submission = merge_with_sample(out_pd)
sample_submission.to_csv('submission.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8640 entries, 2023-05-01 00:00:00 to 2023-07-03 23:45:00
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   absolute_humidity_2m:gm3        8640 non-null   float32
 1   air_density_2m:kgm3             8640 non-null   float32
 2   ceiling_height_agl:m            6243 non-null   float32
 3   clear_sky_energy_1h:J           8640 non-null   float32
 4   clear_sky_rad:W                 8640 non-null   float32
 5   cloud_base_agl:m                7690 non-null   float32
 6   dew_or_rime:idx                 8640 non-null   float32
 7   dew_point_2m:K                  8640 non-null   float32
 8   diffuse_rad:W                   8640 non-null   float32
 9   diffuse_rad_1h:J                8640 non-null   float32
 10  direct_rad:W                    8640 non-null   float32
 11  direct_rad_1h:J                 8640 non-null   float32
 12